# TKET syntax guide

In [54]:
from pytket import Circuit

In [55]:
c = Circuit(2, 2) # define a circuit with 2 qubits and 2 bits
c.H(0)            # add a Hadamard gate to qubit 0
c.Rz(0.25, 0)     # add an Rz gate of angle 0.25*pi to qubit 0
c.CX(1, 0)         # add a CX gate with control qubit 1 and target qubit 0
c.measure_all()   # measure qubits 0 and 1, recording the results in bits 0 and 1

[H q[0]; Rz(0.25) q[0]; CX q[1], q[0]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [56]:
#from jax import numpy as jnp, random, value_and_grad, jit
import matplotlib.pyplot as plt

## Circuit Analysis

In [57]:
from pytket.circuit import Circuit, OpType

In [58]:
c = Circuit(4, name="example1")
c.add_gate(OpType.CU1, 0.5, [0, 1])
c.H(0).X(1).Y(2).Z(3)
c.X(0).CX(1, 2).Y(1).Z(2).H(3)
c.Y(0).Z(1)
c.add_gate(OpType.CU1, 0.5, [2, 3])
c.H(2).X(3)
c.Z(0).H(1).X(2).Y(3).CX(3, 0)

[CU1(0.5) q[0], q[1]; Y q[2]; Z q[3]; H q[0]; X q[1]; H q[3]; X q[0]; CX q[1], q[2]; Y q[0]; Y q[1]; Z q[2]; Z q[0]; Z q[1]; CU1(0.5) q[2], q[3]; H q[1]; H q[2]; X q[3]; X q[2]; Y q[3]; CX q[3], q[0]; ]

In [59]:
c.n_qubits # read number of qubits
c.name # read name
c.depth() # read depth
c.depth_by_type(OpType.CU1), c.depth_by_type(OpType.H) # depth by gate type - counts gates that cannot be parallelized

(2, 2)

In [60]:
from pytket.circuit.display import render_circuit_jupyter

In [61]:
render_circuit_jupyter(c)

In [62]:
from pytket.extensions.qiskit import tk_to_qiskit

In [63]:
print(tk_to_qiskit(c))

               ┌───┐┌───┐┌───┐  ┌───┐             ┌───┐
q_0: ─■────────┤ H ├┤ X ├┤ Y ├──┤ Z ├─────────────┤ X ├
      │U1(π/2) ├───┤└───┘├───┤  ├───┤   ┌───┐     └─┬─┘
q_1: ─■────────┤ X ├──■──┤ Y ├──┤ Z ├───┤ H ├───────┼──
       ┌───┐   └───┘┌─┴─┐├───┤  └───┘   ├───┤┌───┐  │  
q_2: ──┤ Y ├────────┤ X ├┤ Z ├─■────────┤ H ├┤ X ├──┼──
       ├───┤   ┌───┐└───┘└───┘ │U1(π/2) ├───┤├───┤  │  
q_3: ──┤ Z ├───┤ H ├───────────■────────┤ X ├┤ Y ├──■──
       └───┘   └───┘                    └───┘└───┘     


In [64]:
cmds = c.get_commands()  # retrieve list of operations in circuit
print(cmds)

[CU1(0.5) q[0], q[1];, Y q[2];, Z q[3];, H q[0];, X q[1];, H q[3];, X q[0];, CX q[1], q[2];, Y q[0];, Y q[1];, Z q[2];, Z q[0];, Z q[1];, CU1(0.5) q[2], q[3];, H q[1];, H q[2];, X q[3];, X q[2];, Y q[3];, CX q[3], q[0];]


In [65]:
cmd0 = cmds[0]
op0 = cmd0.op
print(op0)
qubits0 = cmd0.args
print(qubits0)

CU1(0.5)
[q[0], q[1]]


In [66]:
op0.get_name()  # normal form
op0.type, op0.params 

(<OpType.CU1: 58>, [0.5])

In [67]:
from pytket.circuit.display import render_circuit_jupyter as draw

In [68]:
circ = Circuit(1, 2) # 3 qubits, 2 classical bits
print(circ.qubits)
print(circ.bits)

[q[0]]
[c[0], c[1]]


In [69]:
from pytket.circuit import Qubit

In [70]:
new_q1 = Qubit("alpha", 0)
new_q2 = Qubit("beta", 2, 1)
circ.add_qubit(new_q1)
circ.add_qubit(new_q2)
print(circ.qubits)

[alpha[0], beta[2, 1], q[0]]


In [71]:
delta_reg = circ.add_q_register("delta", 2) # adding register of qubits in one go
print(circ.qubits)

[alpha[0], beta[2, 1], delta[0], delta[1], q[0]]


In [72]:
circ.CX(delta_reg[0], delta_reg[1])  # add CX gate to control q[0] and target q[1]

[CX delta[0], delta[1]; ]

In [73]:
circ.H(new_q1)
circ.CX(Qubit("q", 0), new_q2)
circ.Rz(0.5, new_q2)

[H alpha[0]; CX q[0], beta[2, 1]; CX delta[0], delta[1]; Rz(0.5) beta[2, 1]; ]

In [74]:
draw(circ)

In [75]:
circ = Circuit(3, 1)
circ.H(0)
circ.CX(0, 1)
circ.CX(1, 2)
circ.Rz(0.25, 2)
circ.Measure(2, 0)
draw(circ)

In [76]:
import numpy as np
from math import sqrt
from pytket.circuit import PauliExpBox, CircBox, ExpBox, Unitary1qBox, Unitary2qBox
from pytket.pauli import Pauli

In [77]:
boxycirc = Circuit(3)
subcirc = Circuit(2, name='Box1') 
subcirc.H(0).Y(1).CX(0,1)
cbox = CircBox(subcirc) # define new box
boxycirc.add_gate(cbox, args=[Qubit(0), Qubit(1)])
draw(boxycirc)

In [78]:
m1 = np.asarray([[1 / 2, sqrt(3) / 2], [sqrt(3) / 2, -1 / 2]]) # 1 qubit unitary circuit box
m1box = Unitary1qBox(m1)
boxycirc.add_unitary1qbox(m1box, 2)
m2 = np.asarray([[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0]]) # 2 qubit unitary circuit box
m2box = Unitary2qBox(m2)
boxycirc.add_unitary2qbox(m2box, 1, 2)
A = np.asarray(
    [[1, 2, 3, 4 + 1j], [2, 0, 1j, -1], [3, -1j, 2, 1j], [4 - 1j, -1, -1j, 1]]
)
ebox = ExpBox(A, 0.5)
boxycirc.add_expbox(ebox, 0, 1)
pbox = PauliExpBox([Pauli.X, Pauli.Z, Pauli.X], 0.75)
boxycirc.add_gate(pbox, [0, 1, 2])

[CircBox q[0], q[1]; Unitary1qBox q[2]; Unitary2qBox q[1], q[2]; ExpBox q[0], q[1]; PauliExpBox q[0], q[1], q[2]; ]

In [79]:
draw(boxycirc)

In [80]:
# alternative syntax for composing serial circuits

c = Circuit(2)
c.CX(0, 1)
c1 = Circuit(2)
c1.CX(0, 1)
c.append(c1)

x, y = Qubit("x"), Qubit("y") # naming qubits
z = Qubit("z")
d = Circuit()
d.add_qubit(z)
d.add_qubit(x)
d.add_qubit(y)
d.CX(x, y)
d.CZ(y, x)
d.CY(y,z)
draw(d)

In [81]:
# Parametrized circuits
from sympy import Symbol

a = Circuit(1)
a.Rz(0.5, 0)
b = Symbol('a')
a.Rz(b, 0)

from pytket.transform import Transform
Transform.RemoveRedundancies().apply(a) # clean up unnecessary gates
a.symbol_substitution({b: 1}) # substitute in parameter values
draw(a)